<a href="https://colab.research.google.com/github/nancy-kataria/NexTrade/blob/main/product_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

=== Imports ===

In [1]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

=== 1. Dataset Download ===

In [2]:
# Download latest version
print("Dowlaod Dataset...")
path = kagglehub.dataset_download("vivek468/superstore-dataset-final")
print(f"Dataset downloaded to: {path}")
csv_file_path = path + "/Sample - Superstore.csv"
print(f"Reading data from: {csv_file_path}")

Dowlaod Dataset...
Dataset downloaded to: /kaggle/input/superstore-dataset-final
Reading data from: /kaggle/input/superstore-dataset-final/Sample - Superstore.csv


=== 2. Load & Clean Data ===

In [3]:
try:
    superstore_data = pd.read_csv(csv_file_path, encoding='ISO-8859-1')
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"ERROR: File not found at {csv_file_path}.")
    exit()

Data loaded successfully.


In [4]:
# Keep necessary columns
columns_to_keep = ['Order ID', 'Order Date', 'Ship Date', 'Customer ID', 'Country', 'City', 'State', 'Postal Code', 'Product ID', 'Product Name', 'Sales', 'Quantity', 'Category', 'Sub-Category']
superstore_data = superstore_data[columns_to_keep]

In [5]:
# Display the first 5 rows to check the data
print("First 5 rows of data:")
print(superstore_data.head())

First 5 rows of data:
         Order ID  Order Date   Ship Date Customer ID        Country  \
0  CA-2016-152156   11/8/2016  11/11/2016    CG-12520  United States   
1  CA-2016-152156   11/8/2016  11/11/2016    CG-12520  United States   
2  CA-2016-138688   6/12/2016   6/16/2016    DV-13045  United States   
3  US-2015-108966  10/11/2015  10/18/2015    SO-20335  United States   
4  US-2015-108966  10/11/2015  10/18/2015    SO-20335  United States   

              City       State  Postal Code       Product ID  \
0        Henderson    Kentucky        42420  FUR-BO-10001798   
1        Henderson    Kentucky        42420  FUR-CH-10000454   
2      Los Angeles  California        90036  OFF-LA-10000240   
3  Fort Lauderdale     Florida        33311  FUR-TA-10000577   
4  Fort Lauderdale     Florida        33311  OFF-ST-10000760   

                                        Product Name     Sales  Quantity  \
0                  Bush Somerset Collection Bookcase  261.9600         2   
1  Hon D

In [6]:
# Convert dates
superstore_data['Order Date'] = pd.to_datetime(superstore_data['Order Date'])
superstore_data['Ship Date'] = pd.to_datetime(superstore_data['Ship Date'])

In [7]:
# drop rows with missing any necessary columns
superstore_data.dropna(subset=columns_to_keep, inplace=True)

=== 3. Precomputation  ===

In [8]:
# 1. Product Popularity
product_popularity = superstore_data.groupby('Product ID').agg({
    'Product Name': 'first',
    'Category': 'first',
    'Sub-Category': 'first',
    'Quantity': 'sum',
    'Sales': 'sum'
}).reset_index()

# Normalize popularity score
product_popularity['popularity_score'] = product_popularity['Quantity'] / product_popularity['Quantity'].max()

# 2. Content-Based Info Preparation
superstore_data['product_info'] = (
    superstore_data['Product Name'].astype(str) + ' ' +
    superstore_data['Category'].astype(str) + ' ' +
    superstore_data['Sub-Category'].astype(str)
)

# One row per product
products = superstore_data.drop_duplicates(subset='Product ID')[
    ['Product ID', 'Product Name', 'Category', 'Sub-Category', 'product_info']
]

# 3. TF-IDF Matrix and Cosine Similarity
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(products['product_info'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 4. Product Index Mapping
product_indices = pd.Series(products.index, index=products['Product ID']).drop_duplicates()

=== 4. Recommendation Functions ===

In [9]:
# === Helper Functions ===
# Get a list of products the customer hasn't purchased yet
# Inputs:
# - customer_id: ID of the target customer
# - df: full transaction data (e.g., superstore_data)
# - product_df: a DataFrame of all products to recommend from (e.g., product_popularity)
# Output:
# - filtered product_df with only unseen products
# - list of purchased Product IDs for fallback logic
def get_unseen_products(customer_id, df, product_df):
    purchased = df[df['Customer ID'] == customer_id]['Product ID'].unique()
    return product_df[~product_df['Product ID'].isin(purchased)], purchased

# Add fallback recommendations if there aren't enough unseen products to recommend
# This uses globally popular products (based on 'Quantity' or 'Sales') to fill the gap
# Inputs:
# - recommendations: filtered list of unseen, ranked products
# - purchased: list of already purchased product IDs
# - product_df: global product list (e.g., product_popularity)
# - n: number of products we want to recommend
# - by: popularity metric ('Quantity' or 'Sales')
# Output:
# - final DataFrame of n recommendations
def add_fallback_if_needed(recommendations, purchased, product_df, n, by):
    if len(recommendations) < n:
        print(f"Customer has only {len(recommendations)} new products available. Showing global popular items instead.")
        fallback = recommend_popular_products(n=n, by=by)
        fallback = fallback[~fallback['Product ID'].isin(purchased)]
        recommendations = pd.concat([recommendations, fallback]).drop_duplicates('Product ID')
    return recommendations

# Get the most common categories and sub-categories for a customer
# This is used in personalized popularity-based recommendation
# Returns two lists: top categories and top sub-categories based on purchase frequency
def get_customer_preferences(customer_id, df):
    customer_data = df[df['Customer ID'] == customer_id]
    if customer_data.empty:
        return [], []
    top_categories = customer_data['Category'].value_counts().index.tolist()
    top_subcategories = customer_data['Sub-Category'].value_counts().index.tolist()
    return top_categories, top_subcategories



# === Recommendation Functions ===
# Recommend top-N globally popular products (ignores customer history)
# Sorts by 'Quantity' (default) or 'Sales'
def recommend_popular_products(n=10, by='Quantity'):
    if by not in ['Quantity', 'Sales']:
        raise ValueError("Parameter 'by' must be either 'Quantity' or 'Sales'")

    return product_popularity.sort_values(by=by, ascending=False).head(n)[['Product ID', 'Product Name', 'Category', 'Sub-Category', by]]

# Unified popularity-based recommendation function
# Modes:
# - No customer_id → returns global top-N products
# - With customer_id, personalized=False → excludes products already purchased
# - With customer_id, personalized=True → filters by customer's preferred categories
def recommend_popular(customer_id=None, personalized=False, n=10, by='Quantity'):
    if by not in ['Quantity', 'Sales']:
        raise ValueError("Parameter 'by' must be either 'Quantity' or 'Sales'")

    if customer_id is None:
        return recommend_popular_products(n, by)

    df = product_popularity

    if personalized:
        top_cats, top_subcats = get_customer_preferences(customer_id, superstore_data)
        if not top_cats or not top_subcats:
            return recommend_popular_products(n, by)
        df = df[(df['Category'].isin(top_cats)) | (df['Sub-Category'].isin(top_subcats))]

    unseen, purchased = get_unseen_products(customer_id, superstore_data, df)
    unseen = unseen.sort_values(by=by, ascending=False)
    final = add_fallback_if_needed(unseen, purchased, product_popularity, n, by)

    return final.head(n)[['Product ID', 'Product Name', 'Category', 'Sub-Category', by]]

# Content-based recommendation using TF-IDF + cosine similarity
# Returns top-N products similar to a given product name (based on 'product_info')
def recommend_similar_products(product_name, top_n=5):
    if product_name not in product_indices:
        return f"Product '{product_name}' not found."

    idx = product_indices[product_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    product_idxs = [i[0] for i in sim_scores]

    return products.iloc[product_idxs][['Product Name', 'Category', 'Sub-Category']]

# Recommend content-similar products to a customer
# Based on the last product the customer purchased
def recommend_to_customer_content_based(customer_id, top_n=5):
    customer_purchases = superstore_data[superstore_data['Customer ID'] == customer_id]
    if customer_purchases.empty:
        return f"No purchase history for customer '{customer_id}'."

    # Get last product bought
    last_product = customer_purchases.sort_values('Order Date', ascending=False).iloc[0]['Product Name']

    print(f"Based on last product: {last_product}")
    return recommend_similar_products(last_product, top_n)

# Hybrid Recommendation (Content-Based + Popularity)
def recommend_hybrid(customer_id, top_n=5, similarity_weight=0.5, popularity_weight=0.5):
    customer_data = superstore_data[superstore_data['Customer ID'] == customer_id]
    if customer_data.empty:
        return "No purchase history found."

    purchased_ids = customer_data['Product ID'].unique()
    purchased_idxs = [product_indices[pid] for pid in purchased_ids if pid in product_indices]
    if not purchased_idxs:
        return "No purchased products found in product index."

    sim_scores = sum(cosine_sim[idx] for idx in purchased_idxs)
    sim_scores = sim_scores / len(purchased_idxs)

    sim_df = pd.DataFrame({
        'product_index': range(len(sim_scores)),
        'similarity_score': sim_scores
    })

    sim_df = sim_df.merge(products.reset_index(), left_on='product_index', right_index=True)
    sim_df = sim_df[~sim_df['Product ID'].isin(purchased_ids)]
    sim_df = sim_df.merge(product_popularity[['Product ID', 'popularity_score']], on='Product ID', how='left')
    sim_df['popularity_score'] = sim_df['popularity_score'].fillna(0)
    sim_df['hybrid_score'] = (
        similarity_weight * sim_df['similarity_score'] +
        popularity_weight * sim_df['popularity_score']
    )

    sim_df = sim_df.sort_values(by='hybrid_score', ascending=False).head(top_n)
    return sim_df[['Product Name', 'Category', 'Sub-Category', 'hybrid_score']]

=== 5. Example Usage ===

In [10]:
print("Top 5 Global Products:")
print(recommend_popular())  # global

print("\n Top 5 Personalized Products (Unseen):")
print(recommend_popular(customer_id='CG-12520'))  # exclude purchased

print("\n Top 5 Personalized Products by Preference:")
print(recommend_popular(customer_id='CG-12520', personalized=True))  # personalized

print("\n Top 5 Content-Based Recommendations:")
print(recommend_to_customer_content_based('CG-12520', 5))

print("\nTop 5 Hybrid Recommendations (Popularity + Content-Based):")
print(recommend_hybrid('CG-12520', top_n=5))  # Combines popularity and content similarity

Top 5 Global Products:
           Product ID                                       Product Name  \
1569  TEC-AC-10003832                 Logitech P710e Mobile Speakerphone   
1144  OFF-PA-10001970                                         Xerox 1881   
694   OFF-BI-10001524  GBC Premium Transparent Covers with Diagonal L...   
721   OFF-BI-10002026                            Avery Arch Ring Binders   
93    FUR-CH-10002647         Situations Contoured Folding Chairs, 4/Set   
325   FUR-TA-10001095                 Chromcraft Round Conference Tables   
1517  TEC-AC-10002049          Logitech G19 Programmable Gaming Keyboard   
835   OFF-BI-10004728  Wilson Jones Turn Tabs Binder Tool for Ring Bi...   
110   FUR-CH-10003774    Global Wood Trimmed Manager's Task Chair, Khaki   
183   FUR-FU-10001473                            DAX Wood Document Frame   

             Category Sub-Category  Quantity  
1569       Technology  Accessories        75  
1144  Office Supplies        Paper        70  

In [ ]:
# visualize top products
top10 = product_popularity.head(10)
plt.figure(figsize=(10,5))
plt.barh(top10['Product Name'], top10['Quantity'], color='skyblue')
plt.gca().invert_yaxis()
plt.xlabel('Total Quantity Sold')
plt.title('Top 10 Most Popular Products')
plt.show()

--- Pre-computation for Recommendations ---

1. Create a unique list of products

2. Create 'product_info' text feature for Content-Based Filtering

3. Calculate TF-IDF Matrix and Cosine Similarity

4. Create a mapping from Product ID to its index in our matrices/products_df

5. Calculate Product Popularity

--- Recommendation Functions ---

1. Popularity-Based Recommendation (Simple)

2. Popularity-Based Recommendation (Personalized with Category)

3. Content-Based Recommendation

4. Hybrid Recommendation (Content Similarity + Popularity)

5. Collaborative Recommendation

6. Hybrid Recommendation 2 (Content Similarity + Popularity + Collaborative)

--- Example Usage ---

--- Visualization ---